#### Prim's algorithm for Minimum Spanning Tree

Prim's algorithm implemented using a modified version of Dijkstra.

In [3]:
class PriorityQueue:
    def __init__(self, capacity=10, items=[]):
        self.empty = (0,'Null')
        if len(items)>0:
            assert capacity >= len(items), "capacity must be greater than or equal to the number of items!"
            self.N = len(items)
            self.queue = [self.empty] + items + [self.empty] * (capacity-self.N)
            self.positions = {item[1]:i for i,item in enumerate(self.queue) if item != self.empty}
            # heapify
            self.heapify()
        else:    
            self.queue = [self.empty] * (capacity+1) 
            self.N = 0

    def heapify(self):
        # start from the last parent node up to root
        for i in range(self.N//2, 0, -1):
            # apply heapify down
            self.heapify_down(i)

    def heapify_up(self, i):
        if i > 1:
            current_item = self.queue[i]
            parent_item = self.queue[i//2]
            if current_item[0] < parent_item[0]:
                # swap with parent
                self.queue[i//2] = current_item
                self.queue[i] = parent_item
                # update positions of swapped items
                self.positions[self.queue[i//2][1]] = i//2
                self.positions[self.queue[i][1]] = i
                # recurse from parent position
                self.heapify_up(i//2) 

    def heapify_down(self, i):
        if 2*i > self.N:
            return 
        elif 2*i < self.N:    
            # get index of child with the smaller key
            if self.queue[2*i][0] <= self.queue[2*i+1][0]:
                j = 2*i
            else:
                j= 2*i+1    
        else:
            j = 2*i
        current_item = self.queue[i]
        if current_item[0] > self.queue[j][0]:
            # swap with smallest child
            self.queue[i] = self.queue[j]
            self.queue[j] = current_item
            # update positions of swapped items
            self.positions[self.queue[i][1]] = i
            self.positions[self.queue[j][1]] = j
            # recurse from child position
            self.heapify_down(j)

    def insert(self, item):
        # insert item into first unoccupied slot
        self.queue[self.N+1] = item
        self.positions[item[1]] = self.N+1
        # heapify up
        self.heapify_up(self.N+1)        
        self.N += 1

    def extract_min(self):
        # swap root with the last item
        min_item = self.queue[1]
        self.positions[min_item[1]] = -1 # set position of removed item to -1
        self.queue[1] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # heapify down
        self.heapify_down(1)
        return min_item

    def delete(self, i):
        current_item = self.queue[i]
        self.positions[current_item[1]] = -1 # set position of removed item to -1
        # swap with last item
        self.queue[i] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # check if we need to heapify up or down
        if current_item[0] < self.queue[i//2][0]:
            self.heapify_up(i)
        else:
            self.heapify_down(i)
        return current_item

    def update_key(self, item_id, new_key):
        # get current position of item 
        i = self.positions[item_id]
        if i == -1:
            # item not in priority queue
            return
        (old_key, _) = self.queue[i]
        # update the item key
        self.queue[i] = (new_key, item_id)
        # check if we need to heapify up or down
        if new_key < old_key:
            self.heapify_up(i)
        else:
            self.heapify_down(i)

    def __str__(self):
        print(f"N = {self.N}")
        return str(self.queue)  

In [4]:
def prim(adjacency_list, verbose=False):
    # pick source node
    s = 'a'
    # initilize minimum edge costs
    d = {v:float('inf') for v in adjacency_list.keys()}
    d[s] = 0
    # intialize parents
    p = {v:None for v in adjacency_list.keys()}
    # insert all nodes priority queue
    Q = PriorityQueue(items=[(d_v, v) for v,d_v in d.items()], capacity=len(d))
    print(Q)
    # initiale subtree vertex set
    S = []
    # intialize tree
    T = []
    
    while len(S) < len(d):
        # extract min
        (d_v, v) = Q.extract_min()
        # add v to explored set
        S.append(v)
        # add edge to tree
        if p[v] is not None:
            T.append((p[v], v))
        # update minimum edge costs for all edges (v,w)
        for (w, l_vw) in adjacency_list[v]:
            if d[w] > l_vw:
                d[w] = l_vw
                # update parent
                p[w] = v
                Q.update_key(w, d[w])
        if verbose:
            print(f"\nFound tree edge {p[v]}-{v} with cost {d[v]}")
            print(f"Updated priority queue: {Q}")        

    # return minimum spanning tree
    return T



In [5]:
# example graph (adjacency_list implemented using dictionary for clarity)
adjacency_list = {'a':[('b',3),('e',6),('f',5)], 'b':[('a',3),('c',1),('f',4)], 'c':[('b',1),('d',6),('f',4)], 'd':[('c',6),('e',8),('f',5)], 'e':[('a',6),('d',8),('f',2)], 'f':[('a',5),('b',4),('c',4),('d',5),('e',2)]}

T = prim(adjacency_list, verbose=True)
print(f"\nMinimum spanning tree: {T}")

N = 6
[(0, 'Null'), (0, 'a'), (inf, 'b'), (inf, 'c'), (inf, 'd'), (inf, 'e'), (inf, 'f')]

Found tree edge None-a with cost 0
N = 5
Updated priority queue: [(0, 'Null'), (3, 'b'), (5, 'f'), (inf, 'c'), (inf, 'd'), (6, 'e'), (0, 'Null')]

Found tree edge a-b with cost 3
N = 4
Updated priority queue: [(0, 'Null'), (1, 'c'), (6, 'e'), (4, 'f'), (inf, 'd'), (0, 'Null'), (0, 'Null')]

Found tree edge b-c with cost 1
N = 3
Updated priority queue: [(0, 'Null'), (4, 'f'), (6, 'e'), (6, 'd'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Found tree edge b-f with cost 4
N = 2
Updated priority queue: [(0, 'Null'), (2, 'e'), (6, 'd'), (5, 'd'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Found tree edge f-e with cost 2
N = 1
Updated priority queue: [(0, 'Null'), (6, 'd'), (0, 'Null'), (5, 'd'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Found tree edge f-d with cost 5
N = 0
Updated priority queue: [(0, 'Null'), (0, 'Null'), (0, 'Null'), (5, 'd'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Minimum spanning tree: [